# Análise dos dados de acervo das bibliotecas da UFRN

In [1]:
import re
import pandas as pd
from os import listdir

## Junção dos datasets

In [27]:
# Variável com os dataframes que iremos usar
dfs = {
    'discentes': None,
    'emprestimos': None,
    'cursos': pd.read_csv('data/cursos-ufrn.csv', sep=';'),
    'exemplares': pd.read_csv('data/exemplares.csv', sep=';')
}

In [2]:
# Arquivos da pasta de datasets
files = listdir('data')
# Regex para cada tipo de dataset que possui vários arquivos
regex = {
    'discentes': re.compile(r'discentes-20[0-9]{2}'),
    'emprestimos': re.compile(r'emprestimos-20[0-9]{3}'),
}
# Filtros aplicados, lista de arquivos que satisfazem os regex
files_filter = {
    'discentes': list(filter(regex['discentes'].search, files)),
    'emprestimos': list(filter(regex['emprestimos'].search, files)),
}
# Mostra ambas as listas de arquivos
print(files_filter)

{'discentes': ['discentes-2012.csv', 'discentes-2017.csv', 'discentes-2013.csv', 'discentes-2010.csv', 'discentes-2018.csv', 'discentes-2014.csv', 'discentes-2015.csv', 'discentes-2016.csv', 'discentes-2009.csv', 'discentes-2011.csv'], 'emprestimos': ['emprestimos-20111.csv', 'emprestimos-20172.csv', 'emprestimos-20021.csv', 'emprestimos-20012.csv', 'emprestimos-20141.csv', 'emprestimos-20081.csv', 'emprestimos-20132.csv', 'emprestimos-20152.csv', 'emprestimos-20061.csv', 'emprestimos-20131.csv', 'emprestimos-20042.csv', 'emprestimos-20091.csv', 'emprestimos-20162.csv', 'emprestimos-20051.csv', 'emprestimos-20002.csv', 'emprestimos-20022.csv', 'emprestimos-20161.csv', 'emprestimos-20121.csv', 'emprestimos-20071.csv', 'emprestimos-20151.csv', 'emprestimos-20122.csv', 'emprestimos-20092.csv', 'emprestimos-20052.csv', 'emprestimos-20171.csv', 'emprestimos-20031.csv', 'emprestimos-20142.csv', 'emprestimos-20112.csv', 'emprestimos-20101.csv', 'emprestimos-20032.csv', 'emprestimos-20062.csv'

In [28]:
def make_df(filenames):
    '''Pega uma lista de arquivos e retorna um único dataframe gerado.'''
    # Lista de dataframes
    dfs = [pd.read_csv('data/'+filename, sep=';') for filename in filenames]
    # Concatena todos os dataframes em um só
    return pd.concat(dfs, ignore_index=True)

# Junta todos os dados de discentes em um unico dataframe
dfs['discentes'] = make_df(files_filter['discentes'])
# O mesmo para emprestimos
dfs['emprestimos'] = make_df(files_filter['emprestimos'])

In [30]:
dfs['emprestimos'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558703 entries, 0 to 2558702
Data columns (total 8 columns):
id_emprestimo           int64
codigo_barras           object
data_renovacao          object
data_emprestimo         object
data_devolucao          object
nome_usuario            object
id_vinculo_usuario      float64
tipo_vinculo_usuario    object
dtypes: float64(1), int64(1), object(6)
memory usage: 156.2+ MB
